<a href="https://colab.research.google.com/github/KILjungjoon/Project_Asian_Parliament/blob/main/Detecting_Singaporean_politician_names_from_Hansard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Install wiki API

In [1]:
!pip install wikipedia-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.5.4-py3-none-any.whl size=13476 sha256=8434646fac6601df1c2f88d361e603dae25538cf6ba0ee266e9421acb5e708bb
  Stored in directory: /root/.cache/pip/wheels/d3/24/56/58ba93cf78be162451144e7a9889603f437976ef1ae7013d04
Successfully built wikipedia-api


In [2]:
import wikipediaapi
wiki=wikipediaapi.Wikipedia('en')

# 2.Read a Singapore Parliamentary Hansard and Pre-processing

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

# the file is the record of Nov.08 in 2006 of the 11th term
html = open("/content/drive/MyDrive/Project_prof.Jang/data/2006_1108.html")
sp = BeautifulSoup(html, features="html.parser")

# kill all script and style elements
for script in sp(["script", "style"]):
    script.extract()    # rip it out

# get text
text = sp.get_text()

# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
text = '\n'.join(chunk for chunk in chunks if chunk)

print(text)

Singapore Parliament Reports - Full Report
A-        A+
Official Report for 2006-11-08Parliament No:11Session No:1Volume No:82Sitting No:2Sitting Date:08-11-2006Column: 97PARLIAMENTARY DEBATESSINGAPOREOFFICIAL REPORTELEVENTH PARLIAMENT
PARTIOF FIRST SESSION
VOLUME82 Wednesday, 8th November, 2006 The House met at1.30 pmPRESENT:Mr SPEAKER (Mr Abdullah Tarmugi (East Coast)).Dr Ahmad Mohd Magad (Pasir Ris-Punggol).Mr Ang Mong Seng (Hong Kah).Mr Baey Yam Keng (Tanjong Pagar).Mr Chan Soo Sen (Joo Chiat).Mr Chiam See Tong (Potong Pasir).Mr Charles Chong (Pasir Ris-Punggol).Mr Christopher de Souza (Holland-Bukit Timah).Dr Fatimah Lateef (Marine Parade).Mr Arthur Fong (West Coast).Mr Cedric Foo Chee Keng (West Coast).Ms Grace Fu Hai Yien (Jurong), Minister of State, Ministry of National Development.Mr Gan Kim Yong (Chua Chu Kang), Minister of State, Ministry of Education and Ministry of Manpower.Mr Goh Chok Tong (Marine Parade), Senior Minister, Prime Minister's Office.Column: 98Mdm Halimah Yac

In [7]:
# pre-processing 2
import re
text1=text.replace("\n", " ")
text2=re.sub("[.,:()]"," ", text1)     # removing non-character signs
text3=re.sub("[\s]{2,3}"," ", text2)   # adjust 2 spaces or more to 1 space
text4=text3.replace("\n", " ")
text4

'Singapore Parliament Reports - Full Report A-   A+ Official Report for 2006-11-08Parliament No 11Session No 1Volume No 82Sitting No 2Sitting Date 08-11-2006Column 97PARLIAMENTARY DEBATESSINGAPOREOFFICIAL REPORTELEVENTH PARLIAMENT PARTIOF FIRST SESSION VOLUME82 Wednesday 8th November 2006 The House met at1 30 pmPRESENT Mr SPEAKER Mr Abdullah Tarmugi East Coast Dr Ahmad Mohd Magad Pasir Ris-Punggol Mr Ang Mong Seng Hong Kah Mr Baey Yam Keng Tanjong Pagar Mr Chan Soo Sen Joo Chiat Mr Chiam See Tong Potong Pasir Mr Charles Chong Pasir Ris-Punggol Mr Christopher de Souza Holland-Bukit Timah Dr Fatimah Lateef Marine Parade Mr Arthur Fong West Coast Mr Cedric Foo Chee Keng West Coast Ms Grace Fu Hai Yien Jurong Minister of State Ministry of National Development Mr Gan Kim Yong Chua Chu Kang Minister of State Ministry of Education and Ministry of Manpower Mr Goh Chok Tong Marine Parade Senior Minister Prime Minister\'s Office Column 98Mdm Halimah Yacob Jurong Mr Hawazi Daipi Sembawang Senior 

# 3.Creating tokens

In [8]:
tokens=text4.split(" ")
print(len(tokens))
tokens

44116


['Singapore',
 'Parliament',
 'Reports',
 '-',
 'Full',
 'Report',
 'A-',
 '',
 '',
 'A+',
 'Official',
 'Report',
 'for',
 '2006-11-08Parliament',
 'No',
 '11Session',
 'No',
 '1Volume',
 'No',
 '82Sitting',
 'No',
 '2Sitting',
 'Date',
 '08-11-2006Column',
 '97PARLIAMENTARY',
 'DEBATESSINGAPOREOFFICIAL',
 'REPORTELEVENTH',
 'PARLIAMENT',
 'PARTIOF',
 'FIRST',
 'SESSION',
 'VOLUME82',
 'Wednesday',
 '8th',
 'November',
 '2006',
 'The',
 'House',
 'met',
 'at1',
 '30',
 'pmPRESENT',
 'Mr',
 'SPEAKER',
 'Mr',
 'Abdullah',
 'Tarmugi',
 'East',
 'Coast',
 'Dr',
 'Ahmad',
 'Mohd',
 'Magad',
 'Pasir',
 'Ris-Punggol',
 'Mr',
 'Ang',
 'Mong',
 'Seng',
 'Hong',
 'Kah',
 'Mr',
 'Baey',
 'Yam',
 'Keng',
 'Tanjong',
 'Pagar',
 'Mr',
 'Chan',
 'Soo',
 'Sen',
 'Joo',
 'Chiat',
 'Mr',
 'Chiam',
 'See',
 'Tong',
 'Potong',
 'Pasir',
 'Mr',
 'Charles',
 'Chong',
 'Pasir',
 'Ris-Punggol',
 'Mr',
 'Christopher',
 'de',
 'Souza',
 'Holland-Bukit',
 'Timah',
 'Dr',
 'Fatimah',
 'Lateef',
 'Marine',
 'Para

# 4.Generating bigram

In [9]:
import pandas as pd
import nltk
bigrams_series = (pd.Series(nltk.ngrams(tokens, 2)).value_counts())
# trigrams_series = (pd.Series(nltk.ngrams(tokens, 3)).value_counts())
# fourgrams_series= (pd.Series(nltk.ngrams(tokens, 4)).value_counts())

bi_unique=bigrams_series.sort_values()   # unique and frequency of bigram

# 5. Generacting a bigram list that starts with an uppercase letter

In [32]:
upper_pair=[]
for grams in bi_unique.index:     # grams is tuple
    if len(grams[0])>0 and len(grams[1])>0 :
        if grams[0][0].isupper()==True and grams[1][0].isupper()==True :
            upper_pair.append(grams[0]+' '+grams[1])

print(upper_pair)

['Hougang In', 'Martin Luther', 'Our GDP', 'Singapore We', 'China That', 'Fund Workfare', 'Singaporeans With', 'Recreation Fund', 'ILO Sir', 'Luther King', 'Prof Yunus', 'Reader’s Forum', 'CDC He', 'Presidential Address', 'Government Cautious', 'The Presidential', 'Internet Why', 'Minister Lim’s', 'Medisave MediShield', 'Medifund I', 'Family Recreation', 'Government Column', 'PAP Government', 'Mao Zedong', 'Minister We', 'Sir Singapore', 'The Secretary-General', 'Our Secretary-General', 'Kang In', 'Sir Mr', 'Chair] ADMINISTRATION', 'ADMINISTRATION OF', 'OF OATH', 'OATH Mr', 'Order Oath', 'Allegiance The', 'Nov30 Nov', 'Singh27 Nov30', 'Inderjit Singh27', 'Geok Choo08', 'Choo08 Nov10', 'Nov10 Nov', 'Yaacob Ibrahim09', 'Ibrahim09 Nov09', 'Nov09 Nov', 'Kuan Yew13', 'Yew13 Nov23', 'Nov23 Nov', 'Hng Kiang14', 'Nov Mdm', 'Kiang14 Nov18', 'Speaker Parliament', 'TARMUGI Speaker', 'Chee Hean04', 'Speaker There', 'Nov15 Nov\xa023', 'Nov26 Nov', 'Hean04 Dec09', 'Dec09 Dec', 'Souza03 Jan12', 'Jan1

# 6. Searching the bigram list on the wiki

In [ ]:
# How to detect s/he is a Singaprean politician
wiki = wikipediaapi.Wikipedia(
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI)

Singaporean_Politicians=[]
for words in upper_pair :
    p_wiki = wiki.page(words)
    if 'Singaporean' in p_wiki.text and 'politician' in p_wiki.text:
        Singaporean_Politicians.append(words)
        # print(words,"☞", p_wiki.text[0:200])

In [35]:
Singaporean_Politicians

['S Iswaran',
 'Hri Kumar',
 'Sylvia Lim',
 'Yaacob Ibrahim',
 'Tharman Shanmugaratnam',
 'Josephine Teo',
 'Lily Neo',
 'Balaji Sadasivan',
 'Zaqy Mohamad',
 'Ellen Lee',
 'Fatimah Lateef',
 'Abdullah Tarmugi',
 'Arthur Fong',
 'Cedric Foo',
 'Grace Fu',
 'K Shanmugam',
 'Jessica Tan',
 'Faishal Ibrahim',
 'Penny Low',
 'Michael Palmer',
 'Cynthia Phua']